In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator  
import random


Generates n random bits (0 or 1) and n random bases (Z-basis or X-basis).

In [11]:
def generate_random_bits_and_bases(n):

    bits = np.random.randint(2, size=n)  # Random 0s and 1s
    bases = np.random.randint(2, size=n)  # Random bases: 0 for Z, 1 for X
    return bits, bases


    Encodes Alice's bits into qubits based on chosen bases.


In [10]:
def encode_qubits(bits, bases):

    qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1) 
        
        if bit == 1:
            qc.x(0) 
        
        if basis == 1:
            qc.h(0)  
        
        qubits.append(qc) 
    
    return qubits


    Simulates quantum transmission and measures qubits in Bob's chosen bases.


In [9]:
def measure_qubits(qubits, bases, backend):

    measured_bits = []
    
    for qc, basis in zip(qubits, bases):
        if basis == 1:
            qc.h(0)  
        
        qc.measure(0, 0) 
        job = backend.run(qc, shots=1)  
        result = job.result()
        counts = result.get_counts()
        measured_bit = int(list(counts.keys())[0]) 
        measured_bits.append(measured_bit)
    
    return measured_bits


    Keeps only the bits where Alice and Bob used the same basis.


In [ ]:
def sift_key(alice_bits, alice_bases, bob_bits, bob_bases):

    key = []
    
    for i in range(len(alice_bits)):
        if alice_bases[i] == bob_bases[i]: 
            key.append(alice_bits[i])
    
    return key


    Runs the complete BB84 protocol.


In [8]:
def bb84_protocol(n, backend):

    # Alice generates bits and bases
    alice_bits, alice_bases = generate_random_bits_and_bases(n)
    
    # Alice encodes and sends qubits
    qubits = encode_qubits(alice_bits, alice_bases)
    
    # Bob chooses bases randomly
    _, bob_bases = generate_random_bits_and_bases(n)
    
    # Bob measures qubits
    bob_bits = measure_qubits(qubits, bob_bases, backend)
    
    # Key sifting: Only keep bits where bases match
    final_key = sift_key(alice_bits, alice_bases, bob_bits, bob_bases)
    
    # Print results
    print("Alice's Bits:   ", alice_bits)
    print("Alice's Bases:  ", alice_bases)
    print("Bob's Bases:    ", bob_bases)
    print("Bob's Measured Bits:", bob_bits)
    print("Final Shared Key: ", final_key)
    
    return final_key


In [ ]:
backend = AerSimulator()  # Quantum Simulator
n_qubits = 10  
shared_key = bb84_protocol(n_qubits, backend)
